In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/one-direction-all-songs-with-lyrics/One_Direction_Proper_Dataset.csv


# Data Preprocessing

In [2]:
import pandas as pd

input_data = pd.read_csv("/kaggle/input/one-direction-all-songs-with-lyrics/One_Direction_Proper_Dataset.csv")
input_data

,S.No.,Song,Artist(s),Writer(s),Album(s),Year,Lyrics
0,1,"""18""",One Direction,"Ed Sheeran, Oliver Frank",Four,2014,[Verse 1: Harry]\nI got a heart and I got a so...
1,2,"""A.M.""",One Direction,"Niall Horan, Liam Payne, Harry Styles, Louis T...",Made in the A.M.,2015,[Intro: Harry]\nWon't you stay till the A.M.?\...
2,3,"""Act My Age""",One Direction,"John Ryan, Julian Bunetta, Ed Drewett",Four,2014,"[Intro]\nOne, two, three, four\n\n[Verse 1: Ni..."
3,4,"""Alive""",One Direction,"Julian Bunetta, John Ryan, Jamie Scott, Louis ...",Midnight Memories,2013,[Verse 1: Liam]\nMy mother told me I should go...
4,5,"""Another World""",One Direction,"Achraf Jannusi, Bilal Hajji, Eric Sanicola, Ge...",Up All Night,2011,"[Verse 1: Liam]\nIt's not me, it's not you, th..."
...,...,...,...,...,...,...,...
90,91,"""Where Do Broken Hearts Go""",One Direction,"Harry Styles, Julian Bunetta, Ruth-Anne Cunnin...",Four,2014,"[Verse 1: Niall, All]\nCounted all my mistakes..."
91,92,"""Why Don't We Go There""",One Direction,"Steve Robson, Claude Kelly, Wayne Hector, Loui...",Midnight Memories,2013,"[Verse 1: Harry Styles]\nI know you want, know..."
92,93,"""Wishing on a Star""",The X Factor Finalists 2011 featuring JLSand O...,Billie Rae Calvin,__,2011,[Chorus: Misha B]\nI'm wishin' on a star\nTo f...
93,94,"""Wolves""",One Direction,"Niall Horan, Liam Payne, Will Champlin, Andrew...",Made in the A.M.,2015,"[Intro: Harry, All]\nI hear them calling for y..."


In [3]:
import pandas as pd
data = pd.read_csv("/kaggle/input/one-direction-all-songs-with-lyrics/One_Direction_Proper_Dataset.csv")["Lyrics"]
lyrics_text = ""
for text in data:
   lyrics_text += text
lyrics_text[:100]

'[Verse 1: Harry]\nI got a heart and I got a soul\nBelieve me, I will use them both\nWe made a start, be'

In [4]:
import re
lyrics_text = re.sub(r'\[.*?\]', '', lyrics_text)
lyrics_text = lyrics_text.lower()
lyrics_text = re.sub(r'[^\w\s]', '', lyrics_text)
lyrics_text[:100]

'\ni got a heart and i got a soul\nbelieve me i will use them both\nwe made a start be it a false one i '

# Tokenization

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(char_level=False)
tokenizer.fit_on_texts([lyrics_text])
total_words = len(tokenizer.word_index) + 1
input_sequences = []
tokens = tokenizer.texts_to_sequences([lyrics_text])[0]
for i in range(1, len(tokens)):
  input_sequences.append(tokens[:i+1])
max_seq_length = 20
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length,padding='pre')
X, y = input_sequences[:, :-1], input_sequences[:, -1]
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_words)

# Model Architecture

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
embedding_dim = 128
lstm_units = 256
model = Sequential([
Embedding(input_dim=total_words, output_dim=embedding_dim,input_length=max_seq_length-1),
LSTM(lstm_units, return_sequences=True),
Dropout(0.2),
LSTM(lstm_units),
Dropout(0.2),
Dense(total_words, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(X, y, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.0432 - loss: 6.0377 - val_accuracy: 0.0570 - val_loss: 5.7951
Epoch 2/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0548 - loss: 5.4631 - val_accuracy: 0.0664 - val_loss: 5.8002
Epoch 3/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0680 - loss: 5.2987 - val_accuracy: 0.0950 - val_loss: 5.6376
Epoch 4/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0988 - loss: 4.9417 - val_accuracy: 0.0968 - val_loss: 5.6016
Epoch 5/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1085 - loss: 4.7476 - val_accuracy: 0.1023 - val_loss: 5.5837
Epoch 6/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1261 - loss: 4.5499 - val_accuracy: 0.0969 - val_loss: 5.6185
Epoch 7/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1374 - loss: 4.3912 - val_accuracy: 0.1019 - val_loss: 5.6461
Epoch 8/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1539 - loss: 4.2578 - val_accu

In [21]:
import numpy as np

def generate_text(seed_text, next_words, model, tokenizer, max_seq_length):
    output_text = seed_text
    line_length = 0  # Track the number of words in the current line
    max_line_length = 10  # Define the maximum number of words per line
    current_singer = None  # Track the current singer for labeling
    section_header = None  # Track section headers (e.g., [Verse 1])

    for _ in range(next_words):
        # Tokenize and pad the seed text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
        
        # Predict the next word
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = tokenizer.index_word.get(predicted[0], "")  # Handle unknown words

        # Detect and format section headers or singer names
        if output_word.startswith("[") and output_word.endswith("]"):
            section_header = output_word  # Update the section header
            output_text += f"\n{section_header}\n"
            line_length = 0
            continue
        
        if ":" in output_word and output_word.endswith("]"):
            current_singer = output_word  # Update the current singer
            output_text += f"\n{current_singer}\n"
            line_length = 0
            continue

        # Add the output word to the text
        if line_length >= max_line_length or output_word == ".":
            output_text += "\n" + output_word  # Add a newline for structure
            line_length = 0
        else:
            output_text += " " + output_word
            line_length += 1
        
        # Update seed text for the next prediction
        seed_text += " " + output_word
    
    return output_text

seed_text = "Beautiful"
generated_text = generate_text(seed_text, 150, model, tokenizer, max_seq_length)
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━